# Gaussian Linear Dynamical System 

In this notebook, we will look at how we can fit a "Gaussian Linear Dynamical System," using the StateSpaceDynamics package. This model is a powerful tool for analyzing time series data and understanding underlying latent processes. For a reference of the details of this model please refer to Chapter 13.3 of Pattern Recognition and Machine Learning by Christopher Bishop.

## Setup

In [15]:
# import the relevant packages
using StateSpaceDynamics
using LinearAlgebra
using Random
using Plots
using LaTeXStrings

## Simulate a Gaussian LDS model

In [ ]:
# create a state-space model for the tutorial
obs_dim = 10
latent_dim = 2

# set up the state parameters
A = 0.95 * [cos(0.25) -sin(0.25); sin(0.25) cos(0.25)] 
Q = Matrix(0.1 * I(2))

x0 = [0.0; 0.0]
P0 = Matrix(0.1 * I(2))

# set up the observation parameters
C = randn(obs_dim, latent_dim)
R = Matrix(0.5 * I(10))


# create the state-space model
true_ssm = GaussianLDS(;A=A, 
                        Q=Q, 
                        C=C, 
                        R=R, 
                        x0=x0, 
                        P0=P0, 
                        obs_dim=obs_dim, 
                        latent_dim=latent_dim, 
                        fit_bool=fill(true, 6))

# simulate data from the model
tSteps = 500
latents, observations = StateSpaceDynamics.sample(true_ssm, tSteps, 1) # one trial for tutorial purposes

In [ ]:
# Create a grid for the vector field
x = y = -3:0.5:3
X = repeat(x', length(y), 1)
Y = repeat(y, 1, length(x))

# Assuming x and y are your grid points
U = zeros(size(X))
V = zeros(size(Y))
for i in 1:size(X, 1)
    for j in 1:size(X, 2)
        v = A * [X[i,j], Y[i,j]]
        U[i,j] = v[1] - X[i,j]  # Change in x
        V[i,j] = v[2] - Y[i,j]  # Change in y
    end
end
# Normalize the vectors for better visualization
magnitude = @. sqrt(U^2 + V^2)
U_norm, V_norm = U ./ magnitude, V ./ magnitude

magnitude = @. sqrt(U^2 + V^2)
U_norm = U ./ magnitude
V_norm = V ./ magnitude

# Plot
p = quiver(X, Y, quiver=(U_norm, V_norm), color=:blue, alpha=0.3,
           linewidth=1, arrow=arrow(:closed, :head, 0.1, 0.1))
plot!(latents[1, :, 1], latents[2, :, 1], xlabel="x₁", ylabel="x₂", color=:black, linewidth=1.5, title="latent dynamics", legend=false)


In [ ]:
# Define parameters
state_dim = 2
obs_dim = 10
time_bins = 500

# Generate sample data
states = latents[:, :, 1]
emissions = observations[:, :, 1]

# Create the main plot
plot(size=(800, 600), layout=@layout[a{0.3h}; b])

# Plot the continuous latent states
lim_states = maximum(abs.(states))
for d in 1:state_dim
    plot!(1:time_bins, states[d, :] .+ lim_states * (d-1), color=:black, linewidth=2, label="", subplot=1)
end
plot!(subplot=1, yticks=(lim_states .* (0:state_dim-1), [L"x_%$d" for d in 1:state_dim]),
      xticks=[], xlims=(0, time_bins), title="Simulated Latent States",
      yformatter=y->"", tickfontsize=12)

# Plot the emissions
lim_emissions = maximum(abs.(emissions))
for n in 1:obs_dim
    plot!(1:time_bins, emissions[n, :] .- lim_emissions * (n-1), color=:black, linewidth=2, label="", subplot=2)
end
plot!(subplot=2, yticks=(-lim_emissions .* (obs_dim-1:-1:0), [L"y_{%$n}" for n in 1:obs_dim]),
      xlabel="time", xlims=(0, time_bins), title="Simulated emissions",
      yformatter=y->"", tickfontsize=12)

# Adjust layout
plot!(link=:x, size=(800, 600), left_margin=10Plots.mm)

# Display the plot
display(current())

# Uncomment to save the figure
# savefig("simulated_latent_states_and_emissions.png")

## Create and fit a Gaussian LDS model to Data

In [ ]:
# create a Gaussian LDS with unknow parameters (except the dimensions)
naive_ssm = GaussianLDS(;obs_dim=obs_dim, latent_dim=latent_dim, fit_bool=fill(true, 6))

# smooth the data to look at the predicted latent states
x_smooth, p_smooth, _ = StateSpaceDynamics.smooth(naive_ssm, observations)

# create a plot to compare the true and predicted latent states
plot()

# plot the true latent states
for d in 1:state_dim
    plot!(1:time_bins, states[d, :] .+ lim_states * (d-1), color=:black, linewidth=2, label="", subplot=1)
end

# plot the predicted latent states
for d in 1:state_dim
    plot!(1:time_bins, x_smooth[d, :, 1] .+ lim_states * (d-1), color=:firebrick, linewidth=2, label="", subplot=1)
end

plot!(subplot=1, yticks=(lim_states .* (0:state_dim-1), [L"x_%$d" for d in 1:state_dim]),
      xticks=[], xlims=(0, time_bins), yformatter=y->"", tickfontsize=12, title="True vs. Predicted Latent States")


In [ ]:
# now fit the LDS using the EM algorithm
elbo, _ = fit!(naive_ssm, observations; max_iter=1000, tol=1e-6)

# smooth the data to look at the predicted latent states
x_smooth, p_smooth, _ = StateSpaceDynamics.smooth(naive_ssm, observations)

# create a plot to compare the true and predicted latent states
plot()

# plot the true latent states
for d in 1:state_dim
    plot!(1:time_bins, states[d, :] .+ lim_states * (d-1), color=:black, linewidth=2, label="", subplot=1)
end

# plot the predicted latent states
for d in 1:state_dim
    plot!(1:time_bins, x_smooth[d, :, 1] .+ lim_states * (d-1), color=:firebrick, linewidth=2, label="", subplot=1)
end

plot!(subplot=1, yticks=(lim_states .* (0:state_dim-1), [L"x_%$d" for d in 1:state_dim]),
      xticks=[], xlims=(0, time_bins), yformatter=y->"", tickfontsize=12, title="True vs. Predicted Latent States")

In [ ]:
plot(elbo, xlabel="iteration", ylabel="ELBO", title="ELBO (Marginal Loglikelihood)", legend=false)

In [ ]:
using BenchmarkTools
using ArrayLayouts

Q = [0.01 0.0; 0.0 0.01]
Q_symmetric = Symmetric(Q)

@btime pinv(Q)


In [ ]:
@btime pinv(Q_symmetric)

In [ ]:
@btime Q \ I(2)

In [ ]:
function Gradient(
    lds::StateSpaceDynamics.LinearDynamicalSystem{S,O}, y::Matrix{T}, x::Matrix{T}
) where {T<:Real,S<:StateSpaceDynamics.GaussianStateModel{T},O<:StateSpaceDynamics.GaussianObservationModel{T}}
    latent_dim, T_steps = size(x)
    obs_dim, _ = size(y)

    A, Q, x0, P0 = lds.state_model.A,
    lds.state_model.Q, lds.state_model.x0,
    lds.state_model.P0
    C, R = lds.obs_model.C, lds.obs_model.R

    inv_R = pinv(R)
    inv_Q = pinv(Q)
    inv_P0 = pinv(P0)

    grad = zeros(T, latent_dim, T_steps)

    # First time step
    grad[:, 1] =
        A' * inv_Q * (x[:, 2] - A * x[:, 1]) + C' * inv_R * (y[:, 1] - C * x[:, 1]) -
        inv_P0 * (x[:, 1] - x0)

    # Middle time steps
    @Threads.threads for t in 2:(T_steps - 1)
        grad[:, t] =
            C' * inv_R * (y[:, t] - C * x[:, t]) - inv_Q * (x[:, t] - A * x[:, t - 1]) +
            A' * inv_Q * (x[:, t + 1] - A * x[:, t])
    end

    # Last time step
    grad[:, T_steps] =
        C' * inv_R * (y[:, T_steps] - C * x[:, T_steps]) -
        inv_Q * (x[:, T_steps] - A * x[:, T_steps - 1])

    return grad
end



In [ ]:
@btime Gradient(naive_ssm, observations[:, :, 1], x_smooth[:, :, 1])